# KNN CLASSIFIER - RANDOM SEARCH

In [1]:
import numpy as np
from scipy.stats import uniform as sp_rand
from sklearn import datasets
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV
import pandas as pd
from datetime import datetime
from time import time

### GLOBAL VARIABLES

In [3]:
DATAPATH = 'data/train_test/'
SEED = 47
NITER = 200
CV = 5
SCORE = 'balanced_accuracy'
usenull = False
NJOBS = -1

### LOAD DATASET

In [4]:
train_features = np.load(DATAPATH+'X_features_train.npy')

In [5]:
train_labels = np.load(DATAPATH+'y_train.npy')

In [13]:
train_features.shape

(1072, 1273)

In [14]:
train_labels.shape

(1225,)

#### Input Null values

In [6]:
if usenull == False:
    train_features[np.isnan(train_features)] = -9999

### TRAIN MODEL

#### Set hyperparameters

In [7]:
# ======== General Parameters ======= #

# Number of neighbors to use.
n_neighbors  = [i for i in range(3,21,2)] 

# Weight function used in prediction. Possible values:
# uniform : uniform weights. All points in each neighborhood are weighted equally.
# distance : weight points by the inverse of their distance. in this case, closer neighbors of a query point 
#            will have a greater influence than neighbors which are further away.
weights = ['uniform', 'distance']

# the distance metric to use for the tree.
metric = ['euclidean', 'minkowski']

# Algorithm used to compute the nearest neighbors:
# - ball_tree will use BallTree
# - kd_tree will use KDTree
# - brute will use a brute-force search.
# - auto will attempt to decide the most appropriate algorithm based on the values passed to fit method.
algorithm = ['auto']

[KNN params](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)

In [8]:
# Create the random grid
random_grid = {
    'n_neighbors' : n_neighbors,
    'weights' : weights,
    'metric' : metric,
    'algorithm': algorithm
}

In [9]:
random_grid

{'n_neighbors': [3, 5, 7, 9, 11, 13, 15, 17, 19],
 'weights': ['uniform', 'distance'],
 'metric': ['euclidean', 'minkowski'],
 'algorithm': ['auto']}

#### Training

In [10]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
model = KNeighborsClassifier()

In [11]:
# Random search of parameters, using CV fold cross validation, 
# search across NITER different combinations, and use all available cores
knn_rsearch = RandomizedSearchCV(estimator = model, param_distributions = random_grid, scoring=SCORE, n_iter = NITER, cv = CV, verbose=2, random_state=SEED, n_jobs = NJOBS)# Fit the random search model


In [12]:
start = time()
knn_rsearch.fit(train_features, train_labels)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), NITER))

ValueError: Found input variables with inconsistent numbers of samples: [1072, 1225]

#### Saving results

In [60]:
cv_results = pd.DataFrame(knn_rsearch.cv_results_)

In [ ]:
cv_results.to_csv('../models/rsearch_knn_classifier_d' + str(datetime.now().date()) + '.csv',sep=';',index=False)

#### Best estimator

In [26]:
knn_rsearch.best_estimator_

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
           metric_params=None, n_jobs=None, n_neighbors=9, p=2,
           weights='distance')

#### Best parameter

In [27]:
knn_rsearch.best_params_

{'weights': 'distance',
 'n_neighbors': 9,
 'metric': 'euclidean',
 'algorithm': 'auto'}

#### Best Score

In [28]:
print(SCORE,' : ', knn_rsearch.best_score_)

balanced_accuracy  :  0.5191704087987988


#### Saving best hyperparameters

In [77]:
np.save('../models/knn_classifier_bestparams_d' + str(datetime.now().date()) + '.npy', knn_rsearch.best_params_)